User pipeline:
- Upload files (assume similar column structure across files)
- Show columns and ask to select (text,numeric, date)
- Continue merging
- Give debug report for missing values that will be removed, words that will be mapped to a certain category, and duplicates. Show exact file and row where it is seen
- Ask user to accept or reject and go back
- If accept proceed to clean and merge
- Create merged downloadable excel link
  

In [8]:
print('hello')

hello


In [9]:
# from rapidfuzz import fuzz,process
import numpy as np
import pandas as pd
from pathlib import Path

Uploads files and check for errors with column headers.

In [10]:
# 📁 Folder containing your test data files
input_folder = Path("../input/")

# 🗂️ Read headers from all Excel/CSV files
file_headers = {}
all_dataframes = []

for file in input_folder.glob("*"):
    if file.suffix.lower() not in [".csv", ".xlsx"]:
        continue
    try:
        if file.suffix.lower() == ".xlsx":
            df = pd.read_excel(file, engine="openpyxl")
        else:
            df = pd.read_csv(file)

        df.columns = [col.strip().title() for col in df.columns]
        file_headers[file.name] = set(df.columns)
        df["__source_file__"] = file.name  # optional: track source
        df['row_number'] = df.index + 2
        all_dataframes.append(df)
    except Exception as e:
        print(f"❌ Could not read {file.name}: {e}")

# 🧠 Analyze headers
header_sets = list(file_headers.values())

if not header_sets:
    print("❗ No valid files found.")
    exit()

common_cols = set.intersection(*header_sets)
all_cols = set.union(*header_sets)

unique_cols_per_file = {
    fname: cols - common_cols
    for fname, cols in file_headers.items()
}

# 📊 Show results
print("\n✅ Common Columns Across All Files:")
print(sorted(common_cols))

print("\n🧩 Unique Columns Per File:")
for fname, unique in unique_cols_per_file.items():
    if unique:
        print(f"- {fname}: {sorted(unique)}")

# ❓ Ask user what to do
decision = input("\nDo you want to [F]ix files and retry or [C]ontinue with common columns? ").strip().lower()

if decision == 'f':
    print("🔁 Please edit your files and run the script again.")
    exit()

# 🧬 Merge only on common columns
merged = pd.concat(
    [df[list(common_cols) + ['__source_file__','row_number']] for df in all_dataframes],
    ignore_index=True
)





✅ Common Columns Across All Files:
['Branch', 'Date', 'Product', 'Quantity']

🧩 Unique Columns Per File:
- sales_2024-03.xlsx: ['Revenue']
- sales_2024-02.xlsx: ['Revenue']
- sales_2024-01.xlsx: ['Rev']


KeyboardInterrupt: Interrupted by user

In [13]:
from pathlib import Path
import pandas as pd

# 📁 Folder containing your test data files
input_folder = Path("../input/")

# 🗂️ Read headers from all Excel/CSV files
file_headers = {}
dataframes_by_file = {}

for file in input_folder.glob("*"):
    if file.suffix.lower() not in [".csv", ".xlsx"]:
        continue
    try:
        if file.suffix.lower() == ".xlsx":
            df = pd.read_excel(file, engine="openpyxl")
        else:
            df = pd.read_csv(file)

        df.columns = [col.strip().title() for col in df.columns]
        file_headers[file.name] = set(df.columns)
        df["__source_file__"] = file.name
        df["row_number"] = df.index + 2
        dataframes_by_file[file.name] = df
    except Exception as e:
        print(f"❌ Could not read {file.name}: {e}")

# 🧠 First pass to find common & unique columns
header_sets = list(file_headers.values())
if not header_sets:
    print("❗ No valid files found.")
    exit()

common_cols = set.intersection(*header_sets)
all_cols = set.union(*header_sets)

unique_cols_per_file = {
    fname: cols - common_cols
    for fname, cols in file_headers.items()
}

# 🧩 Ask user about unique columns
print("\n🧩 Found unique columns:")
for fname, uniques in unique_cols_per_file.items():
    if not uniques:
        continue
    print(f"\n📄 {fname}")
    for col in sorted(uniques):
        print(f"  • {col}")
        suggestion = input(f"    ↪ Rename '{col}' to match another column? Enter name or leave blank to skip: ").strip()
        if suggestion:
            # Rename in the actual DataFrame and update header record
            df = dataframes_by_file[fname]
            if col in df.columns:
                df.rename(columns={col: suggestion}, inplace=True)
                file_headers[fname].remove(col)
                file_headers[fname].add(suggestion)

# 🔁 Recompute common columns after renaming
header_sets = list(file_headers.values())
common_cols = set.intersection(*header_sets)
all_dataframes = list(dataframes_by_file.values())

if not common_cols:
    print("❌ No common columns found even after renaming.")
    exit()

# 🧬 Merge only on common columns
merged = pd.concat(
    [df[list(common_cols) + ['__source_file__', 'row_number']] for df in all_dataframes],
    ignore_index=True
)

# ✅ Show results
print("\n✅ Final Common Columns Across All Files:")
print(sorted(common_cols))

print("\n✅ Merged Data Preview:")
print(merged.head())


❌ Could not read ~$sales_2024-01.xlsx: File is not a zip file

🧩 Found unique columns:

📄 sales_2024-03.xlsx
  • Revenue


    ↪ Rename 'Revenue' to match another column? Enter name or leave blank to skip:  



📄 sales_2024-02.xlsx
  • Revenue


    ↪ Rename 'Revenue' to match another column? Enter name or leave blank to skip:  



📄 sales_2024-01.xlsx
  • Balls


    ↪ Rename 'Balls' to match another column? Enter name or leave blank to skip:  


  • Rev


    ↪ Rename 'Rev' to match another column? Enter name or leave blank to skip:  Revenue



✅ Final Common Columns Across All Files:
['Branch', 'Date', 'Product', 'Quantity', 'Revenue']

✅ Merged Data Preview:
    Branch  Quantity       Date  Revenue  Product     __source_file__  \
0      NYC         4 2024-03-01   125.92  T-Shirt  sales_2024-03.xlsx   
1  Chicago         3 2024-03-01    98.13      Hat  sales_2024-03.xlsx   
2  Chicago         2 2024-03-01    77.36      Hat  sales_2024-03.xlsx   
3  Chicago         4 2024-03-01    96.67    Socks  sales_2024-03.xlsx   
4  Chicago         2 2024-03-01    37.17      Hat  sales_2024-03.xlsx   

   row_number  
0           2  
1           3  
2           4  
3           5  
4           6  


In [14]:
merged.shape

(878, 7)

Figure out data types.

In [15]:


# 🔍 Infer data types using Pandas
print("\n📊 Inferred Data Types:")
dtype_map = {}
conversion_funcs = {
    "datetime": pd.to_datetime,
    "numeric": pd.to_numeric,
    "string": lambda x: x.astype(str)
}

for col, dtype in merged.dtypes.items():
    if col in ["__source_file__", "row_number"]:
        continue
    if pd.api.types.is_datetime64_any_dtype(dtype):
        inferred = "datetime"
    elif pd.api.types.is_numeric_dtype(dtype):
        inferred = "numeric"
    else:
        inferred = "string"
    dtype_map[col] = inferred
    print(f"- {col}: {inferred}")

# ✍️ Ask user to confirm or override
print("\n🛠️ Adjust data types if needed:")
print("Type one of: string, numeric, datetime. Leave blank to accept default.\n")

for col, default_type in dtype_map.items():
    user_input = input(f"{col} [{default_type}]: ").strip().lower()
    if user_input in conversion_funcs:
        dtype_map[col] = user_input

# ✅ Apply conversions
print("\n🔄 Converting columns...")
for col, dtype in dtype_map.items():
    if col in ["__source_file__", "row_number"]:
        continue
    try:
        if dtype == "string":
            merged[col] = merged[col].astype(str)
        elif dtype == "numeric":
            merged[col] = pd.to_numeric(merged[col], errors="coerce")
        elif dtype == "datetime":
            merged[col] = pd.to_datetime(merged[col], errors="coerce")
        print(f"✔️ {col} converted to {dtype}")
    except Exception as e:
        print(f"⚠️ Could not convert {col} to {dtype}: {e}")



📊 Inferred Data Types:
- Branch: string
- Quantity: numeric
- Date: datetime
- Revenue: numeric
- Product: string

🛠️ Adjust data types if needed:
Type one of: string, numeric, datetime. Leave blank to accept default.



Branch [string]:  
Quantity [numeric]:  
Date [datetime]:  
Revenue [numeric]:  
Product [string]:  



🔄 Converting columns...
✔️ Branch converted to string
✔️ Quantity converted to numeric
✔️ Date converted to datetime
✔️ Revenue converted to numeric
✔️ Product converted to string


Check for mispellings in each string column.

In [18]:
# dtype_map
merged['Quantity']

0      4
1      3
2      2
3      4
4      2
      ..
873    2
874    1
875    1
876    1
877    2
Name: Quantity, Length: 878, dtype: int64

In [25]:

def detect_numeric_outliers(data, iqr_multiplier=1.5):
    numeric_cols = data.select_dtypes(include='number').columns
    outlier_mask = pd.Series(False, index=data.index)

    for col in numeric_cols:
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - iqr_multiplier * iqr
        upper_bound = q3 + iqr_multiplier * iqr

        col_outliers = (data[col] < lower_bound) | (data[col] > upper_bound)
        outlier_mask |= col_outliers  # Combine masks across columns

    outlier_indices = data.index[outlier_mask]
    return outlier_indices


merged.loc[detect_numeric_outliers(merged)]

,Branch,Quantity,Date,Revenue,Product,__source_file__,row_number
68,NYC,4,2024-03-07,159.32,Socks,sales_2024-03.xlsx,70
205,NYC,4,2024-03-22,159.05,Hat,sales_2024-03.xlsx,207
308,LA,4,2024-02-01,159.24,Socks,sales_2024-02.xlsx,5
380,NYC,4,2024-02-10,158.92,Hoodie,sales_2024-02.xlsx,77
491,NYC,4,2024-02-21,159.70,Socks,sales_2024-02.xlsx,188


In [6]:
def fuzzy_clean_text_columns(df: pd.DataFrame, dtype_map: dict, threshold: int = 30) -> pd.DataFrame:
    """
    For string columns in dtype_map, find fuzzy matches above a threshold and ask the user how to resolve.
    - If user says values are the same, prompt for a unified replacement.
    - If user says values are different, prompt for individual replacements.
    """
    for col in dtype_map:
        if dtype_map[col] != "string":
            continue

        print(f"\n🔍 Checking column: {col}")
        fixed_values = {}

        while True:
            unique_vals = df[col].dropna().unique().tolist()
            unique_vals = sorted(set(unique_vals) - set(fixed_values.keys()), key=lambda x: str(x).lower())

            pair_found = False
            for i, val in enumerate(unique_vals):
                matches = process.extract(val, unique_vals, scorer=fuzz.ratio, limit=None)
                similar = [(other, score) for other, score, _ in matches if other != val and score >= threshold and other not in fixed_values]

                for other, score in similar:
                    pair_found = True
                    val_loc = df[df[col] == val][["__source_file__", "row_number"]].iloc[0]
                    other_loc = df[df[col] == other][["__source_file__", "row_number"]].iloc[0]

                    print(f"\n🧐 Found similar values in '{col}':")
                    print(f"  • '{val}'  → {val_loc['__source_file__']} (row {val_loc['row_number']})")
                    print(f"  • '{other}' → {other_loc['__source_file__']} (row {other_loc['row_number']})")
                    print(f"  Similarity: {score}%")

                    decision = input("Do these refer to the same thing? [Y]es / [N]o / [S]kip: ").strip().lower()

                    if decision == 'y':
                        replacement = input(f"What should both '{val}' and '{other}' be changed to? [default: '{val}']: ").strip()
                        if not replacement:
                            replacement = val
                        df[col] = df[col].replace({val: replacement, other: replacement})
                        fixed_values[val] = replacement
                        fixed_values[other] = replacement
                        print(f"✅ Replaced both with '{replacement}'")
                        break  # rerun from start after update

                    elif decision == 'n':
                        new_val = input(f"What should '{val}' be changed to? [Enter to keep]: ").strip()
                        if new_val:
                            df[col] = df[col].replace(val, new_val)
                            fixed_values[val] = new_val
                            print(f"✅ '{val}' replaced with '{new_val}'")
                        else:
                            fixed_values[val] = val  # Mark as seen

                        new_other = input(f"What should '{other}' be changed to? [Enter to keep]: ").strip()
                        if new_other:
                            df[col] = df[col].replace(other, new_other)
                            fixed_values[other] = new_other
                            print(f"✅ '{other}' replaced with '{new_other}'")
                        else:
                            fixed_values[other] = other  # Mark as seen
                        break  # rerun from start after update

                    else:
                        print("⏭️ Skipped.")
                        fixed_values[val] = val
                        fixed_values[other] = other
                        break  # Continue to next match

                if pair_found:
                    break  # Rerun with updated values

            if not pair_found:
                print(f"✅ No more fuzzy matches found in '{col}'.")
                break

    return df

In [7]:
fuzzy_clean_text_columns(merged,dtype_map,30)


🔍 Checking column: Product
✅ No more fuzzy matches found in 'Product'.

🔍 Checking column: Branch

🧐 Found similar values in 'Branch':
  • 'LA'  → sales_2024-03.xlsx (row 7)
  • 'Lac' → sales_2024-01.xlsx (row 3)
  Similarity: 40.0%


Do these refer to the same thing? [Y]es / [N]o / [S]kip:  Y
What should both 'LA' and 'Lac' be changed to? [default: 'LA']:  LA


✅ Replaced both with 'LA'

🧐 Found similar values in 'Branch':
  • 'Las'  → sales_2024-01.xlsx (row 7)
  • 'Law' → sales_2024-01.xlsx (row 2)
  Similarity: 66.66666666666667%


Do these refer to the same thing? [Y]es / [N]o / [S]kip:  N
What should 'Las' be changed to? [Enter to keep]:  LA


✅ 'Las' replaced with 'LA'


What should 'Law' be changed to? [Enter to keep]:  LA


✅ 'Law' replaced with 'LA'

🧐 Found similar values in 'Branch':
  • 'New York'  → sales_2024-01.xlsx (row 5)
  • 'NYC' → sales_2024-03.xlsx (row 2)
  Similarity: 36.36363636363637%


Do these refer to the same thing? [Y]es / [N]o / [S]kip:  Y
What should both 'New York' and 'NYC' be changed to? [default: 'New York']:  NYC


✅ Replaced both with 'NYC'
✅ No more fuzzy matches found in 'Branch'.


,Product,Revenue,Branch,Quantity,Date,__source_file__,row_number
0,T-Shirt,125.92,NYC,4,2024-03-01,sales_2024-03.xlsx,2
1,Hat,98.13,Chicago,3,2024-03-01,sales_2024-03.xlsx,3
2,Hat,77.36,Chicago,2,2024-03-01,sales_2024-03.xlsx,4
3,Socks,96.67,Chicago,4,2024-03-01,sales_2024-03.xlsx,5
4,Hat,37.17,Chicago,2,2024-03-01,sales_2024-03.xlsx,6
...,...,...,...,...,...,...,...
873,Hat,44.70,LA,2,2024-01-31,sales_2024-01.xlsx,303
874,Socks,20.39,Chicago,1,2024-01-31,sales_2024-01.xlsx,304
875,T-Shirt,23.79,NYC,1,2024-01-31,sales_2024-01.xlsx,305
876,Socks,36.67,Chicago,1,2024-01-31,sales_2024-01.xlsx,306


Check for duplicates and allow user to decide how to proceed

In [8]:
def resolve_duplicates(data: pd.DataFrame, row_id_col: str = "row_number") -> pd.DataFrame:
    # Columns to check for duplicates (exclude row_id_col if present)
    cols_to_check = [col for col in data.columns if col != row_id_col]

    # Find all exact duplicates based on relevant columns
    duplicate_mask = data.duplicated(subset=cols_to_check, keep=False)
    duplicates = data[duplicate_mask].copy()
    non_duplicates = data[~duplicate_mask].copy()

    if duplicates.empty:
        print("✅ No duplicates found.")
        return data

    # Assign a group number to each duplicate set
    duplicates["group_num"] = (
        duplicates.groupby(cols_to_check, sort=False).ngroup() + 1
    )

    cleaned_rows = []

    for group_id, group_df in duplicates.groupby("group_num", sort=False):
        print(f"\n🔍 Duplicate Group #{group_id}")
        print(group_df.to_string(index=False))

        # Prompt user
        while True:
            decision = input("Keep all rows (K) or remove duplicates (R)? ").strip().lower()
            if decision in {"k", "r"}:
                break
            print("Please enter 'K' to keep all rows or 'R' to remove duplicates.")

        if decision == "k":
            cleaned_rows.append(group_df.drop(columns="group_num"))
        else:
            cleaned_rows.append(group_df.iloc[[0]].drop(columns="group_num"))

    # Combine cleaned rows with non-duplicates
    final_df = pd.concat([*cleaned_rows, non_duplicates], ignore_index=True)

    return final_df


In [9]:
result = resolve_duplicates(merged)
result


🔍 Duplicate Group #1
Product  Revenue  Branch  Quantity       Date    __source_file__  row_number  group_num
T-Shirt     40.7 Chicago         2 2024-01-01 sales_2024-01.xlsx           8          1
T-Shirt     40.7 Chicago         2 2024-01-01 sales_2024-01.xlsx           9          1


Keep all rows (K) or remove duplicates (R)?  R



🔍 Duplicate Group #2
Product  Revenue Branch  Quantity       Date    __source_file__  row_number  group_num
 Hoodie    52.16    NYC         4 2024-01-02 sales_2024-01.xlsx          16          2
 Hoodie    52.16    NYC         4 2024-01-02 sales_2024-01.xlsx          27          2
 Hoodie    52.16    NYC         4 2024-01-02 sales_2024-01.xlsx          31          2
 Hoodie    52.16    NYC         4 2024-01-02 sales_2024-01.xlsx          34          2


Keep all rows (K) or remove duplicates (R)?  R


,Product,Revenue,Branch,Quantity,Date,__source_file__,row_number
0,T-Shirt,40.70,Chicago,2,2024-01-01,sales_2024-01.xlsx,8
1,Hoodie,52.16,NYC,4,2024-01-02,sales_2024-01.xlsx,16
2,T-Shirt,125.92,NYC,4,2024-03-01,sales_2024-03.xlsx,2
3,Hat,98.13,Chicago,3,2024-03-01,sales_2024-03.xlsx,3
4,Hat,77.36,Chicago,2,2024-03-01,sales_2024-03.xlsx,4
...,...,...,...,...,...,...,...
869,Hat,44.70,LA,2,2024-01-31,sales_2024-01.xlsx,303
870,Socks,20.39,Chicago,1,2024-01-31,sales_2024-01.xlsx,304
871,T-Shirt,23.79,NYC,1,2024-01-31,sales_2024-01.xlsx,305
872,Socks,36.67,Chicago,1,2024-01-31,sales_2024-01.xlsx,306
